In [2]:
%matplotlib inline 
import pandas as pd
import numpy as np
import xlrd
import openpyxl

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import math

In [4]:
import _pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# データの読み込み

## CSVファイル

In [5]:
df = pd.read_csv("csvs/zansa.csv", names=["ts", "lat", "lng", "text"])

In [6]:
df

,ts,lat,lng,text
0,2016-08-11 21:56:33,35.465031,139.485946,NaN
1,2016-08-11 22:03:55,35.464851,139.485781,NaN
2,2016-08-11 22:14:59,35.341035,139.488140,moge
3,2016-10-03 09:56:37,35.395429,139.449904,テスト
4,2016-10-04 11:49:13,35.395455,139.449633,NaN
5,2016-10-05 06:45:59,35.342667,139.468346,送信テスト 10月4日分本雑紙未回収 鵠沼神明3-9-16 和田様
6,2016-10-05 06:48:34,35.414385,139.475290,送信テスト 10月4日分 カン未回収 高倉1515-1 佐藤様 43-6982
7,2016-10-05 09:31:00,35.395768,139.449851,test
8,2016-10-05 10:06:31,35.392561,139.466889,2-486 残渣 カーペット
9,2016-10-05 10:08:34,35.388926,139.466587,2-449 マット類


### 3次地域メッシュの列を追加

In [7]:
def calc_mesh3 (dlat, dlng):
    mc_1_la_p = dlat * 60 // 40
    mc_1_ln_u = (dlng - 100) // 1
    mc_2_la_q = dlat * 60 % 40 // 5
    mc_2_ln_v = ((dlng - 100) % 1) * 60 // 7.5
    mc_3_la_r = dlat * 60 % 40 % 5 * 60 // 30
    mc_3_ln_w = ((dlng - 100) % 1) * 60 % 7.5 * 60 // 45
    mc = int(mc_1_la_p * 1000000 + mc_1_ln_u * 10000 + mc_2_la_q * 1000 + mc_2_ln_v * 100 + mc_3_la_r * 10 + mc_3_ln_w)
    
    return mc

In [8]:
mesh3 = []
for i in range(len(df)):
    mesh3.append(calc_mesh3(df.lat[i],df.lng[i]))

In [9]:
df['mesh3'] = mesh3

### 外れ値を消去

In [11]:
df = df.drop([1118,1281])

In [12]:
len(df.mesh3)

1552

In [13]:
df.index = range(1552)

In [14]:
print(df.mesh3[1]*10 + 1)

533913581


#### メッシュコードを緯度経度に変換する関数

In [15]:
import re  # 正規表現によるパターンマッチ

def meshcode_to_latlng(code, loc = 'C'):

	if type(code) != int:  # 整数型に（文字列が渡された場合に対応）
		try:
			code = int(code)
		except:  # 形式チェック
			print("Error in  mesh_lib.meshocode_to_latlong()  Not a valid code handed.")
			raise

	code = str(code) # あらためて文字型に。

	code12 = ''
	code34 = ''
	code5  = ''
	code6  = ''
	code7  = ''
	code8  = ''

	loc = loc.upper()

	if re.match("^C|(NE)|(NW)|(SE)|(SW)$", loc) == None:
		raise Exception ("Invalid option to meshcode_to_latlong()")

	match_1_result = re.match("\d{4}", code)

	if re.match("\d{4}", code):  # 最初の４文字が数字

		code12 = code[0:2]
		code34 = code[2:4]
		lat_width  = 2.0 / 3.0  # grid cell の緯度方向の間隔
		long_width = 1.0        # grid cell の経度方向の間隔

	else:
		return (None)           # メッシュコードとして無効

	if re.match("\d{6}", code):   # 少なくとも最初の６文字は数字

		code5 = code[4:5]
		code6 = code[5:6]
		lat_width  /= 8.0;
		long_width /= 8.0;

	if re.match("\d{8}", code):  # 最初の８文字は数字

		code7 = code[6:7]
		code8 = code[7:8]
		lat_width  /= 10.0;
		long_width /= 10.0;

    # 以下、南西コーナーの座標を求める。

	lat  = float(code12) * 2 / 3          #  一次メッシュ
	long = float(code34) + 100

	if (code5 != '') & (code6 != ''):     # 二次メッシュ or 三次メッシュ
		lat  += (float(code5) * 2 / 3) / 8
		long += float(code6) / 8 

	if (code7 != '') & (code8 != ''):     # 三次メッシュ
		lat  += float(code7) * 2 / 3 / 8 / 10
		long += float(code8) / 8 / 10 

	# ここまでで南東端の緯度・経度が得られた

	# 中央の座標なら、一区画の幅（経度幅）・高さ（緯度幅）の半分を足す。
	if loc == 'C':
		lat  += lat_width  / 2 
		long += long_width / 2

	# 北端の座標なら、一区画の高さ（緯度幅）を足す
	if re.search('N', loc):
		lat += lat_width

	# 東端の座標なら、一区画の幅（経度幅）を足す
	if re.search('E', loc):
		long  += long_width

	return (lat, long)     # タプルを返す。

#### 緯度経度をdmsに変換

In [16]:
def deg_to_s(in_deg):

	deg 		= int(in_deg)
	rest_in_deg = in_deg - deg

	min 		= int( rest_in_deg * 60 )
	rest_in_min = rest_in_deg * 60 - min;

	sec = rest_in_min * 60;

	# もし繰り上がってしまっていたら、その分の調整、

	if sec >= 60:	# 秒が60を越えたら
		sec -= 60
		min += 1 


	if min == 60:	 # 分が60 を越えたら
	   min -= 60
	   deg += 1

	return (sec)	# タプルを返す。

#### 1/2,1/4,1/8 のメッシュコードを判定する関数

In [17]:
def calc_mesh_small (mlat, lat, mlng, lng, div_l):
    p_lat = 0
    p_lng = 0

    if div_l == 12:
        p_lat = 15 / 3600
        p_lng = 22.5 / 3600
    elif div_l == 14:
        p_lat = 7.5 / 3600
        p_lng = 11.25 / 3600
    elif div_l == 18:
        p_lat = 3.75 / 3600
        p_lng = 5.625 / 3600
    
    cond_lat_u = mlat + p_lat <= lat < mlat + 2 * p_lat
    cond_lat_d = mlat <= lat < mlat + p_lat
    cond_lng_l = mlng <= lng < mlng + p_lng
    cond_lng_r = mlng + p_lng <= lng < mlng + 2 * p_lng
    
    #print(mlat, mlat+p_lat, mlat+2*p_lat, lat)
    #print(mlng, mlng+p_lng, mlng+2*p_lng, lng)
    
    if cond_lat_d and cond_lng_l:
        return 1
        print(1)
    elif cond_lat_d and cond_lng_r:
        return 2
        print(2)
    elif cond_lat_u and cond_lng_l:
        return 3
        print(3)
    elif cond_lat_u and cond_lng_r:
        return 4
        print(4)
    else:
        return 0
        print(0)

In [18]:
for index, row in df.iterrows():
    try:
        mlat, mlng = meshcode_to_latlng(row["mesh3"], "SW")
        #print(mlat,mlng)
        code_12 = calc_mesh_small(mlat, row["lat"], mlng, row["lng"], 2)
        #print(code_12)
    except:
        pass 

## 3次メッシュとテキストデータを行列にする

In [19]:
col_mesh3 = df.mesh3

In [20]:
col_text = df.text

In [21]:
df_m3_t = pd.concat([col_mesh3, col_text], axis = 1)

In [22]:
m3_t_array = df_m3_t.as_matrix()

### 被りなしのmesh3のリストを作成

In [23]:
m3_array = col_mesh3.as_matrix()

In [24]:
m3_array

array([53391358, 53391358, 53390309, ..., 52397386, 52397387, 52397396])

In [25]:
m3_set = set(m3_array)

In [26]:
m3_list = list(m3_set)
# 集合を使いやすくするために配列に変換し直している
with open('pkls/m3_list.pkl', "wb") as f:
    _pickle.dump(m3_list, f)

### item名のリスト作成

In [27]:
list_item = ["布団類", "じゅうたん類", "鞄類", "座布団", "枕", "クッション","靴", "イス", "ダンボール", "傘", "プラスチック類","家電製品", "発泡スチロール"]

In [28]:
import _pickle
with open('pkls/list_item.pkl', 'wb') as f:
    _pickle.dump(list_item, f)

### mk dataframe(index=m3_list, column=list_item)

In [29]:
len(list_item)
matrix2 = np.random.rand(len(m3_list), len(list_item))

In [30]:
matrix1 = np.random.rand(len(m3_list), len(list_item)).astype('int64')
matrix1

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [40]:
df_m3_item = pd.DataFrame(matrix1, index=m3_list, columns=list_item)

### df_m3_itemのデータの中身を作成

In [43]:
df_m3_item.columns

Index(['布団類', 'じゅうたん類', '鞄類', '座布団', '枕', 'クッション', '靴', 'イス', 'ダンボール', '傘',
       'プラスチック類', '家電製品', '発泡スチロール'],
      dtype='object')

In [44]:
# for i in range(len(df_m3_item)):
#     count = 0
#     for j in range(len(df)):
#         if df_m3_item.index[i] == df.mesh3[j]:
#             print (df_m3_item.index[i])
#         if str(df_m3_item.columns[i]) in str(df.text[j]):
#             print(str(df_m3_item.columns[i]))
#         if df_m3_item.index[i] == df.mesh3[j] and str(df_m3_item.columns[i]) in str(df.text[j]):
#             count += 1
#             print(count)
#         df_m3_item[i] = count
#         print(count)

In [45]:
def count_zansa():
    for columns, line in df_m3_item.iteritems():
        for index, row in df_m3_item.iterrows():
            count = 0

            if columns == "布団類":
                condition = "布団" or "ふとん" or "ベッド" or "ベット" or "マットレス"
                n_condition = "座布団"
            elif columns == "じゅうたん類":
                condition = "カーペット" or "じゅうたん" or "ジュータン" or "絨毯" or "マット"
                n_condition = "マットレス"
            elif columns == "鞄類":
                condition = "かばん" or "バッグ" or "リュック" or "鞄"
            elif columns == "座布団":
                condition = "座布団"
            elif columns == "枕":
                condition = "枕" or "まくら" or "マクラ"
            elif columns == "クッション":
                condition = "クッション" or "くっしょん"
            elif columns == "靴":
                condition = "靴" or "くつ" or "クツ" or "サンダル"
            elif columns == "イス":
                condition = "イス" or "いす" or "椅子" or "座椅子" or "座いす"
            elif columns == "ダンボール":
                condition = "ダンボール" or "段ボール"
            elif columns == "傘":
                condition = "傘" or "かさ" or "カサ"
            elif columns == "プラスチック類":
                condition = "プラ"
            elif columns == "家電製品":
                condition = "電" or "機"
                n_condition = "電" and "機"
            elif columns == "発泡スチロール":
                condition = "発砲" or "ハッポー"

            for ind, r in df.iterrows():
                if index == r["mesh3"] and condition in str(r["text"]):
                    count += 1
                elif n_condition:
                    if index == r["mesh3"] and n_condition in str(r["text"]):
                        count -= 1
                if count < 0:
                    count = 0
            row[columns] = count
            print (row[columns])

In [46]:
# import _pickle
# with open('df_m3_item.', 'wb') as f:
#     _pickle.dump(df_m3_item, f)

In [31]:
with open('pkls/df_m3_item.pkl', 'rb') as f:
    df_m3_item_p = _pickle.load(f)

In [32]:
df_m3_item_p

,布団類,じゅうたん類,鞄類,座布団,枕,クッション,靴,イス,ダンボール,傘,プラスチック類,家電製品,発泡スチロール
53390336,5,2,0,1,0,3,1,0,0,0,1,0,0
53390337,12,1,0,2,5,5,4,0,0,0,0,0,1
53390338,1,0,0,0,0,0,0,0,0,0,0,0,0
53390344,2,3,0,0,0,4,0,0,0,0,0,0,0
53390345,12,6,0,5,3,5,0,0,0,0,0,0,0
53390346,8,4,0,2,1,0,0,0,0,0,0,1,0
53390347,6,0,0,0,2,2,0,0,0,0,0,0,0
53390348,1,0,0,1,3,0,0,0,0,0,0,0,0
53390352,0,0,0,0,0,0,0,0,0,0,0,0,0
53390353,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
df_m3_item_p.to_csv(path_or_buf='csvs/matrix_mesh_kind.csv', sep=',', encoding='utf_16')

### 行列分解実行

In [33]:
mat_m3_item = df_m3_item_p.as_matrix()

In [34]:
# def mf(R, K, alpha=0.0002, beta=0.02, step=10000):
#     P = np.random.rand(K,R.shape[0])
#     Q = np.random.rand(K,R.shape[1])
#     e_ui = 0
#     errs = []
#     for s in range(step):
#         for u in range(R.shape[0]):
#             for i in range(R.shape[1]):
#                 if R[u,i] == 0:
#                     continue
#                 e_ui = R[u,i] - np.dot(P[:,u],Q[:,i])
#                 for k in range(K):
#                     P[k,u] += alpha * (2 * e_ui * Q[k,i] - beta * P[k,u])
#                     Q[k,i] += alpha * (2 * e_ui * P[k,u] - beta * Q[k,i])
#         error_R = np.dot(P.T,Q)
#         e = 0
#         for u in range(R.shape[0]):
#             for i in range(R.shape[1]):
#                 if R[u,i] == 0:
#                     continue
#                 e += pow(R[u,i] - np.dot(P[:,u],Q[:,i]),2)
#                 for k in range(K):
#                     e += 0.5 * beta * (pow(P[k][u],2) + pow(Q[k][i],2))
#         errs.append(e)
#         if s % 100 == 0 :
#             print (e)
#     return P,Q, errs

In [35]:
# p,q, errs = mf(mat_m3_item, 3)

In [36]:
# np.dot(p.T,q

In [37]:
# np.sum(pow(mat_m3_item - np.dot(p.T,q),2))

In [38]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.plot(list(range(len(errs))), errs)

In [39]:
# mat_m3_item

In [40]:
# np.save('mat_m3_item.npy', mat_m3_item)

## ガウス・ポアソン分布を用いた行列分解

In [51]:
class MF(nn.Module):
    def __init__(self, input_size, K): 
        super(MF,self).__init__()
        self.input_size = input_size
        self.P = nn.Parameter(torch.Tensor(K, input_size[0]).uniform_(0,1))
        self.Q = nn.Parameter(torch.Tensor(K, input_size[1]).uniform_(0,1))
        
    def forward(self):
        return torch.matmul(torch.t(self.P), self.Q) 

    def project(self):
        self.P.data = torch.max(self.P.data, torch.zeros(self.P.data.size()))
        self.Q.data = torch.max(self.Q.data, torch.zeros(self.Q.data.size()))

In [52]:
def ll_gaussian(x, mu, sigma2, missing=False):
    if missing:
        weight = (x != 0).float()
        return -0.5 * (math.log(2 * math.pi * sigma2) + torch.pow(x - mu, 2) / sigma2) * weight
    return -0.5 * (math.log(2 * math.pi * sigma2) + torch.pow(x - mu, 2) / sigma2)

In [53]:
def ll_poisson(x, lam, missing=False):
    if missing:
        weight = (x != 0).float()
        return (x * torch.log(lam) - lam) * weight
    return x * torch.log(lam) - lam

In [54]:
def update_param_gaussian(R, model, optimizer, missing=False):
    optimizer.zero_grad()
    pred = model()
    log_likelihood = ll_gaussian(R, pred, 1, missing=missing)
    prior_P = ll_gaussian(model.P, 0, 1/0.003)
    prior_Q = ll_gaussian(model.Q, 0, 1/0.003)
    loss = -(torch.sum(log_likelihood) + torch.sum(prior_P) + torch.sum(prior_Q))
    loss.backward()
    optimizer.step()
    return loss.data[0]

In [55]:
def update_param_poisson(R, model, optimizer, missing=False):
    optimizer.zero_grad()
    pred = model()
    log_likelihood = ll_poisson(R, pred, missing=missing)
    loss = -torch.sum(log_likelihood)
    loss.backward()
    optimizer.step()
    return loss.data[0]


In [56]:
# mat_m3_item_tensorf = torch.from_numpy(mat_m3_item).float()

In [57]:
# mat_m3_item_tensorf

In [58]:
# if __name__ == '__main__':
#     R = Variable(mat_m3_item_tensorf)

#     model = MF(R.size(), 2)
#     optimizer_p = optim.Adam([model.P], lr = 0.0001)
#     optimizer_q = optim.Adam([model.Q], lr = 0.0001)

#     for i in range(8000):
#         update_param_poisson(R, model, optimizer_p, missing=True)
#         model.project()
#         loss = update_param_poisson(R, model, optimizer_q, missing=True)
#         model.project()

#         if i % 1000 == 0:
#             print("Epoch {}: Loss:{}".format(i+1, loss))

#     print("Epoch {}: Loss:{}".format(i+1, loss))
#     print("Ground Truth:{}".format(R.data))
#     print("Predict:{}".format(model().data))

#     print("P:{}".format(model.P.data))
#     print("Q:{}".format(model.Q.data))

#     poi_loss = loss
#     poi_Predict = Variable(model().data)
#     poi_P = Variable(model.P.data)
#     poi_Q = Variable(model.Q.data)

In [59]:
# MSE_pmf = torch.sum(torch.pow(R - poi_Predict,2))
# print(MSE_pmf)

In [60]:
# if __name__ == '__main__':
#     R = Variable(mat_m3_item_tensorf)

#     model = MF(R.size(), 6)
#     optimizer_p = optim.Adam([model.P], lr = 0.02)
#     optimizer_q = optim.Adam([model.Q], lr = 0.02)

#     for i in range(1000):
#         update_param_gaussian(R, model, optimizer_p, missing=True)
#         model.project()
#         loss = update_param_gaussian(R, model, optimizer_q, missing=True)
#         model.project()

#         if i % 100 == 0:
#             print("Epoch {}: Loss:{}".format(i+1, loss))

#     print("Epoch {}: Loss:{}".format(i+1, loss))
#     print("Ground Truth:{}".format(R.data))
#     print("Predict:{}".format(model().data))

#     print("P:{}".format(model.P.data))
#     print("Q:{}".format(model.Q.data))
    
#     gau_loss = loss
#     gau_Predict = Variable(model().data)
#     gau_P = Variable(model.P.data)
#     gau_Q = Variable(model.Q.data)


In [61]:
# MSE_gmf = torch.sum(torch.pow(R - gau_Predict,2))
# print(MSE_gmf)

In [62]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.plot(list(range(len(loss))), loss)

In [69]:
# sns.heatmap(mat_m3_item, cmap='autumn')

## 1/2以下の地域メッシュのコードを作成してdfに突っ込む

In [76]:
for i in [12,14,18]:
    print(i)

12
14
18


In [77]:
for i in [12,14,18]:
    if i == 12:
        mesh = "mesh3"
    if i == 14:
        mesh = "mesh12"
    if i == 18:
        mesh = "mesh14"
    for index, row in df.iterrows():
        try:
            mlat, mlng = meshcode_to_latlng(row["mesh3"], "SW")
            #print(mlat,mlng)
            code = calc_mesh_small(mlat, row["lat"], mlng, row["lng"], i)
            #print(code_12)
        except:
            pass 
        
    if i == 12:
        code_12 = code
    if i == 14:
        code_14 = code
    if i == 18:
        code_18 = code

In [78]:
def input_mcode_to_df(mcode):
    mesh_12_array = []
    mesh_14_array = []
    mesh_18_array = []
    if mcode == 12:
        marray = mesh_12_array
        mesh = "mesh12"
        code = code_12
    if mcode == 14:
        marray = mesh_14_array
        mesh = "mesh14"
        code = code_14
    if mcode == 18:
        marray = mesh_18_array
        mesh = "mesh18"
        code = code_18
        
    for index, row in df.iterrows():
        mlat, mlng = meshcode_to_latlng(row[mesh], "SW")
        code = calc_mesh_small(mlat, row["lat"], mlng, row["lng"], mcode)
        result = row[mesh] * 10 + code
        marray.append(result)
        
    if mcode == 12:
        code = code_12
        marray = mesh_12_array
        df_m12 = pd.DataFrame({'mesh12':mesh_12_array})
        df["mesh12"] = df_m12
    if mcode == 14:
        code = code_14
        marray = mesh_14_array
        df_m14 = pd.DataFrame({'mesh14':mesh_14_array})
        df["mesh14"] = df_m14
    if mcode == 18:
        code = code_18
        marray = mesh_18_array
        df_m18 = pd.DataFrame({'mesh18':mesh_18_array})
        df["mesh18"] = df_m18

In [79]:
input_mcode_to_df(12)
input_mcode_to_df(14)
input_mcode_to_df(18)

KeyError: 'mesh12'

In [ ]:
mesh_12_array

## 1/2次メッシュとテキストデータを行列にする

In [ ]:
col_mesh12 = df.mesh12

In [ ]:
col_mesh12

In [ ]:
df_m12_t = pd.concat([col_mesh12, col_text], axis = 1)

In [ ]:
m12_t_array = df_m12_t.as_matrix()

In [ ]:
m12_t_array

### 被りなしのmesh12のリストを作成

In [ ]:
m12_array = col_mesh12.as_matrix()

In [ ]:
m12_array

In [ ]:
m12_set = set(m12_array)

In [ ]:
m12_set
# 被りをなくすために集合に変換している

In [ ]:
m12_list = list(m12_set)
# 集合を使いやすくするために配列に変換し直している

In [ ]:
len(m12_list)

### mk dataframe(index=m3_list, column=list_item)

In [ ]:
len(list_item)
matrix2 = np.random.rand(len(m3_list), len(list_item))

In [ ]:
matrix1 = np.random.rand(len(m3_list), len(list_item)).astype('int64')
matrix1

In [ ]:
df_m3_item = pd.DataFrame(matrix1, index=m3_list, columns=list_item)

In [ ]:
len(list_item)

### df_m3_itemのデータの中身を作成

In [ ]:
df_m3_item.columns

In [ ]:
def input_element(input_mat, mcode):
    if mcode = 3:
        mesh = "mesh3"
    elif mcode = 12:
        mesh = "mesh12"
    elif mcode = 14:
        mesh = "mesh14"
    elif mcode = 18:
        mesh = "mesh18"
        
    for columns, line in input_mat.iteritems():
        for index, row in input_mat.iterrows():
            count = 0

            if columns == "布団類":
                condition = "布団" or "ふとん" or "ベッド" or "ベット" or "マットレス"
                n_condition = "座布団"
            elif columns == "じゅうたん類":
                condition = "カーペット" or "じゅうたん" or "ジュータン" or "絨毯" or "マット"
                n_condition = "マットレス"
            elif columns == "鞄類":
                condition = "かばん" or "バッグ" or "リュック" or "鞄"
            elif columns == "座布団":
                condition = "座布団"
            elif columns == "枕":
                condition = "枕" or "まくら" or "マクラ"
            elif columns == "クッション":
                condition = "クッション" or "くっしょん"
            elif columns == "靴":
                condition = "靴" or "くつ" or "クツ" or "サンダル"
            elif columns == "イス":
                condition = "イス" or "いす" or "椅子" or "座椅子" or "座いす"
            elif columns == "ダンボール":
                condition = "ダンボール" or "段ボール"
            elif columns == "傘":
                condition = "傘" or "かさ" or "カサ"
            elif columns == "プラスチック類":
                condition = "プラ"
            elif columns == "家電製品":
                condition = "電" or "機"
                n_condition = "電" and "機"
            elif columns == "発泡スチロール":
                condition = "発砲" or "ハッポー"

            for ind, r in df.iterrows():
                if index == r[mesh] and condition in str(r["text"]):
                    count += 1
                elif n_condition:
                    if index == r[mesh] and n_condition in str(r["text"]):
                        count -= 1
                if count < 0:
                    count = 0
            row[columns] = count
            print (row[columns])